In [1]:
#NO REGULARISATION
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
import numpy as np
from sklearn.utils import shuffle
import pandas as pd 
from skimage.io import imread, imshow
from skimage.transform import rescale
from os import listdir
from sklearn.model_selection import train_test_split as tts
from skimage.color import rgb2hsv
from skimage.filters import threshold_otsu
from os import listdir
import pickle

In [2]:
 tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
with open('otsudataX','rb') as f:
    X = pickle.load(f)

with open('otsudataY','rb') as g:
    Y = pickle.load(g)


In [4]:
X_train, X_test, y_train, y_test = tts(X, Y, test_size=0.33)

In [5]:
X_train = X_train.reshape(6671,120,160,1)
X_train.shape
y_train[0:20]

array([0, 2, 3, 1, 2, 2, 1, 0, 3, 0, 0, 3, 1, 1, 2, 3, 0, 2, 1, 3])

In [6]:
y_train_cate = keras.utils.to_categorical(
    y_train,num_classes=4,dtype='float32'
)

y_train_cate[0:10]

array([[1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.]], dtype=float32)

In [7]:
model = keras.Sequential([
    #cnn layers
    keras.layers.Conv2D(filters=32,kernel_size=(3, 3),activation='relu',input_shape=(120,160,1)),
    keras.layers.MaxPooling2D((2,2)),

    keras.layers.Conv2D(filters=32,kernel_size=(3, 3),activation='relu'),
    keras.layers.MaxPooling2D((2,2)),

    keras.layers.Conv2D(filters=32,kernel_size=(3, 3),activation='relu'),
    keras.layers.MaxPooling2D((2,2)),

    keras.layers.Conv2D(filters=64,kernel_size=(3, 3),activation='relu'),
    keras.layers.MaxPooling2D((2,2)),

    #dense normal ann
    keras.layers.Flatten(), 
    keras.layers.Dropout(0.78),   
    keras.layers.Dense(90,activation='relu'),
    keras.layers.Dense(60,activation='relu'),
    keras.layers.Dense(30,activation='relu'),
    keras.layers.Dense(10,activation='relu'),
    
    keras.layers.Dense(4,activation='softmax'),
    #softmax normalises the probability of the output set
])

model.compile(
    optimizer='adam',
    loss = 'categorical_crossentropy',
    metrics=['accuracy']
)

model.fit(X_train,y_train_cate,epochs=90)

Epoch 1/90
209/209 [==============================] - 13s 19ms/step - loss: 1.3817 - accuracy: 0.2587
Epoch 2/90
209/209 [==============================] - 4s 18ms/step - loss: 1.2255 - accuracy: 0.4127
Epoch 3/90
209/209 [==============================] - 5s 22ms/step - loss: 1.0979 - accuracy: 0.4816
Epoch 4/90
209/209 [==============================] - 5s 26ms/step - loss: 1.0296 - accuracy: 0.5148
Epoch 5/90
209/209 [==============================] - 5s 26ms/step - loss: 0.9778 - accuracy: 0.5423
Epoch 6/90
209/209 [==============================] - 5s 26ms/step - loss: 0.9569 - accuracy: 0.5464
Epoch 7/90
209/209 [==============================] - 5s 26ms/step - loss: 0.9135 - accuracy: 0.5684
Epoch 8/90
209/209 [==============================] - 6s 27ms/step - loss: 0.8850 - accuracy: 0.5782
Epoch 9/90
209/209 [==============================] - 6s 30ms/step - loss: 0.8445 - accuracy: 0.5995
Epoch 10/90
209/209 [==============================] - 6s 27ms/step - loss: 0.8302 - accur

In [8]:
X_test = X_test.reshape(3286,120,160,1)
y_test_cate = keras.utils.to_categorical(
    y_test,num_classes=4,dtype='float32'
)

In [9]:
model.evaluate(X_test,y_test_cate)

103/103 [==============================] - 2s 14ms/step - loss: 0.6732 - accuracy: 0.7432


[0.6731948852539062, 0.7431527972221375]

In [10]:
yprob = model.predict(X_test)

In [11]:
yprobcl = [np.argmax(i) for i in yprob]

In [12]:
yprobcl[0:10]

[1, 0, 1, 0, 1, 3, 1, 1, 3, 0]

In [13]:
y_test[0:10]

array([0, 2, 1, 0, 1, 0, 1, 1, 2, 0])

In [14]:
from sklearn.metrics import accuracy_score

In [15]:
accuracy_score(y_test,yprobcl)

0.7431527693244065

In [16]:
from sklearn.metrics import confusion_matrix

In [18]:
from sklearn.utils.multiclass import unique_labels
def plot(y_true,y_pred):    
    column = [f'Predicted {label}' for label in unique_labels(y_true)]
    indexx = [f'Actual {label}' for label in unique_labels(y_true)]
    table = pd.DataFrame(confusion_matrix(y_true,y_pred),columns=column,index=indexx)
    return table

plot(y_test,yprobcl)

,Predicted 0,Predicted 1,Predicted 2,Predicted 3
Actual 0,481,24,99,239
Actual 1,43,663,67,25
Actual 2,44,15,740,43
Actual 3,161,21,63,558
